# [CPT12] Competition 1 - Order Brushing

Using Pandas Loc  
Score: 0.88950

In [1]:
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('Copy of Copy of order_brush_order.csv')
df.event_time = df.event_time.astype('datetime64')

In [3]:
df

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [8]:
from tqdm import tqdm

In [42]:
predictions = {}
for shopid, df_shop in tqdm(df.groupby('shopid'), total=df.shopid.nunique()):
    if df_shop.shape[0] < 3:
        predictions[shopid] = '0'
        continue
        
    df_shop.sort_values('event_time', inplace=True)
    df_shop.set_index('event_time', inplace=True)
    
    suspected = pd.DataFrame()
    for _, row in df_shop.iterrows():
        start_time = row.name
        end_time = start_time + dt.timedelta(hours=1)
        
        df_orders = df_shop.loc[start_time:end_time]
        conc_rate = len(df_orders)/df_orders.userid.nunique()
        
        if conc_rate >= 3:
            suspected = suspected.append(
                df_orders.userid.value_counts().reset_index()
            )
    
    if len(suspected) == 0:
        predictions[shopid] = '0'
        continue
    
    suspected = (suspected
                 .groupby('index')['userid']
                 .sum()
                 .sort_values(ascending=False))
    top_suspect = suspected.iloc[0]
    tagged_users = []
    for user, count in suspected.items():
        if count == top_suspect:
            tagged_users.append(user)
        else:
            break
    if len(tagged_users) > 1:
        pred = '&'.join(str(i) for i in sorted(tagged_users))
    else:
        pred = str(tagged_users[0])
    predictions[shopid] = pred       

  0%|          | 0/18770 [00:00<?, ?it/s]/Users/aven/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
100%|██████████| 18770/18770 [01:56<00:00, 161.34it/s] 


In [44]:
new_pred_df = (pd.DataFrame
               .from_dict(predictions, orient='index')
               .reset_index()
               .rename({'index':'shopid', 0:'userid'},
                       axis=1)
              )
new_pred_df.to_csv('submission4.csv', index=False)

In [45]:
new_pred_df[new_pred_df.userid!='0']

,shopid,userid
40,10402,77819
57,10536,672345
111,42472,740844
114,42818,170385453
129,76934,190449497
...,...,...
17401,203531250,114282846
17960,204225676,198662175
18155,208696908,214111334
18557,210197928,52867898
